In [1]:
import pandas as pd

import numpy as np

In [71]:
annotation_droplet = pd.read_csv('/home/ubuntu/tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_droplet.csv')
annotation_droplet.columns = annotation_droplet.columns.str.replace('.', "_")
print(annotation_droplet.shape)
annotation_droplet.head()

(55656, 20)


/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cell,cell_ontology_class,cell_ontology_id,channel,cluster_ids,free_annotation,mouse_id,mouse_sex,subsetA,subsetA_cluster_ids,subsetB,subsetB_cluster_ids,subsetC,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
0,10X_P7_8_AAACGGGAGGATATAC,myeloid cell,CL:0000763,10X_P7_8,20,dendritic cells and interstital macrophages,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,17.024721,-32.902836
1,10X_P7_8_AAACGGGTCTCGTATT,alveolar macrophage,CL:0000583,10X_P7_8,5,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,25.160619,25.066566
2,10X_P7_8_AAAGATGCAGATCTGT,B cell,CL:0000236,10X_P7_8,12,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,1.740567,46.488878
3,10X_P7_8_AAATGCCAGATAGTCA,natural killer cell,CL:0000623,10X_P7_8,7,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,-31.647934,-2.208061
4,10X_P7_8_AAATGCCCAAACTGCT,T cell,CL:0000084,10X_P7_8,21,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,-37.281266,-5.619565


In [72]:
droplet_channels = annotation_droplet[['channel', 'tissue', 'mouse_id']].drop_duplicates()
print(droplet_channels.shape)
droplet_channels

(28, 3)


,channel,tissue,mouse_id
0,10X_P7_8,Lung,3-F-56
621,10X_P7_9,Lung,3-F-57
2141,10X_P8_12,Lung,3-M-5/6
2925,10X_P8_13,Lung,3-M-7/8
5449,10X_P7_2,Marrow,3-F-56
7181,10X_P7_3,Marrow,3-F-57
9101,10X_P4_0,Tongue,3-M-8
12202,10X_P4_1,Tongue,3-M-9
14690,10X_P7_10,Tongue,3-F-56
16639,10X_P4_7,Spleen,3-M-8


In [73]:
bladder_channel = '10X_P4_3'
lung_channel = '10X_P7_8'

bladder_lung_channels = bladder_channel, lung_channel

In [104]:
! aws s3 cp s3://czbiohub-maca/10x_data/$lung_channel/barcodes.tsv - | wc -l

625


In [74]:
droplet_channels.query('channel in @bladder_lung_channels')

,channel,tissue,mouse_id
0,10X_P7_8,Lung,3-F-56
38084,10X_P4_3,Bladder,3-M-8


So let's just do mouse `3-M-8` to compare with kidney and spleen 

In [94]:
spleen_kidney = 'Spleen', 'Kidney'

bladder_mouse_id = '3-M-8'

samples = droplet_channels.query('(tissue in @spleen_kidney) and (mouse_id in @bladder_mouse_id)')
samples

,channel,tissue,mouse_id
16639,10X_P4_7,Spleen,3-M-8
45065,10X_P4_5,Kidney,3-M-8


In [101]:
! aws s3 ls --human-readable s3://czbiohub-maca/10x_data/10X_P4_5/

2017-08-22 06:34:07   67.6 MiB 10X_P4_5.mus.cell-gene.csv
2017-08-22 06:31:40   15.9 GiB 10X_P4_5.tgz
2017-08-31 21:49:57   11.3 KiB barcodes.tsv
2017-08-31 21:50:01  339.8 KiB genes.tsv
2017-08-31 21:50:05   15.1 MiB matrix.mtx
2017-08-22 06:20:19  612 Bytes metrics_summary.csv
2018-06-19 02:40:45   15.8 GiB possorted_genome_bam.bam
2018-06-19 02:40:46    4.4 MiB possorted_genome_bam.bam.bai
2017-09-15 22:04:17    9.3 MiB raw_gene_bc_matrices_h5.h5
2017-08-22 06:20:18    2.5 MiB web_summary.html


In [102]:
! aws s3 cp s3://czbiohub-maca/10x_data/10X_P4_5/barcodes.tsv - | wc -l

610


In [103]:
! aws s3 cp s3://czbiohub-maca/10x_data/10X_P4_7/barcodes.tsv - | wc -l

3458


## Write `barcodes_filtered.tsv` to be only the barcodes used in the annotation

In [95]:
channel = samples.channel.iloc[0]
channel

'10X_P4_7'

In [98]:
annotation_droplet['barcode'] = annotation_droplet['cell'].str.split('_').str[-1] + '-1'
annotation_droplet.head()

,cell,cell_ontology_class,cell_ontology_id,channel,cluster_ids,free_annotation,mouse_id,mouse_sex,subsetA,subsetA_cluster_ids,...,subsetB_cluster_ids,subsetC,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,barcode
0,10X_P7_8_AAACGGGAGGATATAC,myeloid cell,CL:0000763,10X_P7_8,20,dendritic cells and interstital macrophages,3-F-56,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Lung,17.024721,-32.902836,AAACGGGAGGATATAC-1
1,10X_P7_8_AAACGGGTCTCGTATT,alveolar macrophage,CL:0000583,10X_P7_8,5,NaN,3-F-56,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Lung,25.160619,25.066566,AAACGGGTCTCGTATT-1
2,10X_P7_8_AAAGATGCAGATCTGT,B cell,CL:0000236,10X_P7_8,12,NaN,3-F-56,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Lung,1.740567,46.488878,AAAGATGCAGATCTGT-1
3,10X_P7_8_AAATGCCAGATAGTCA,natural killer cell,CL:0000623,10X_P7_8,7,NaN,3-F-56,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Lung,-31.647934,-2.208061,AAATGCCAGATAGTCA-1
4,10X_P7_8_AAATGCCCAAACTGCT,T cell,CL:0000084,10X_P7_8,21,NaN,3-F-56,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Lung,-37.281266,-5.619565,AAATGCCCAAACTGCT-1


In [96]:
annotation_channel = annotation_droplet.query('channel == @channel')
print(annotation_channel.shape)
annotation_channel.head()

(3439, 20)


,cell,cell_ontology_class,cell_ontology_id,channel,cluster_ids,free_annotation,mouse_id,mouse_sex,subsetA,subsetA_cluster_ids,subsetB,subsetB_cluster_ids,subsetC,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
16639,10X_P4_7_AAACCTGAGAAGGACA,B cell,CL:0000236,10X_P4_7,1,NaN,3-M-8,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spleen,-3.405185,18.483613
16640,10X_P4_7_AAACCTGAGCTAAGAT,natural killer cell,CL:0000623,10X_P4_7,10,NaN,3-M-8,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spleen,-22.856869,-31.439583
16641,10X_P4_7_AAACCTGCAACAACCT,B cell,CL:0000236,10X_P4_7,1,NaN,3-M-8,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spleen,0.889944,28.197329
16642,10X_P4_7_AAACCTGCAGCCAATT,B cell,CL:0000236,10X_P4_7,1,NaN,3-M-8,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spleen,-1.247326,36.036188
16643,10X_P4_7_AAACCTGCAGCTCCGA,B cell,CL:0000236,10X_P4_7,1,NaN,3-M-8,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spleen,-10.226957,18.533754


In [ ]:
annnotation

In [77]:
! aws s3 ls --human-readable s3://czbiohub-maca/10x_data/10X_P4_7/

2017-08-22 12:46:51  324.6 MiB 10X_P4_7.mus.cell-gene.csv
2017-08-22 12:42:56   21.9 GiB 10X_P4_7.tgz
2017-08-31 21:56:04   64.2 KiB barcodes.tsv
2017-08-31 22:00:17  339.8 KiB genes.tsv
2017-08-31 22:04:32   51.9 MiB matrix.mtx
2017-08-22 12:26:46  616 Bytes metrics_summary.csv
2018-06-19 02:48:08   21.7 GiB possorted_genome_bam.bam
2018-06-19 02:48:08    5.7 MiB possorted_genome_bam.bam.bai
2017-09-15 22:24:33   15.2 MiB raw_gene_bc_matrices_h5.h5
2017-08-22 12:26:45    2.5 MiB web_summary.html


In [100]:
barcodes_original = pd.read_csv('s3://czbiohub-maca/10x_data/10X_P4_7/barcodes.tsv', index_col=0, header=None)
print(barcodes_original.shape)
barcodes_original.head()

(3458, 0)


""
0
AAACCTGAGAAGGACA-1
AAACCTGAGCTAAGAT-1
AAACCTGCAACAACCT-1
AAACCTGCAGCCAATT-1
AAACCTGCAGCTCCGA-1


In [78]:
s3_prefix = 's3://czbiohub-maca/10x_data/'

In [87]:
ksizes = ",".join(map(str, (21, 27, 33, 51)))
num_hashes = 5000

In [88]:
output_prefix = f's3://olgabot-maca/10x/sourmash_compute/ksizes={ksizes}_num_hashes={num_hashes}/'
output_prefix

's3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=5000/'

In [89]:
samples['ksizes'] = ksizes
samples['num_hashes'] = num_hashes
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,channel,tissue,mouse_id,ksizes,num_hashes,tenx,id,output
16639,10X_P4_7,Spleen,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_7,Spleen_10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.sig"
45065,10X_P4_5,Kidney,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_5,Kidney_10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_5.sig"


In [90]:
samples['tenx'] = s3_prefix + samples['channel']
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,channel,tissue,mouse_id,ksizes,num_hashes,tenx,id,output
16639,10X_P4_7,Spleen,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_7,Spleen_10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.sig"
45065,10X_P4_5,Kidney,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_5,Kidney_10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_5.sig"


In [91]:
samples['id'] = samples.apply(lambda x: '{tissue}_{channel}'.format(**x), axis=1)
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,channel,tissue,mouse_id,ksizes,num_hashes,tenx,id,output
16639,10X_P4_7,Spleen,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_7,Spleen_10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.sig"
45065,10X_P4_5,Kidney,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_5,Kidney_10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_5.sig"


In [92]:
pd.options.display.max_colwidth = 1000

In [93]:
samples['output'] = output_prefix + samples['id'] + '.sig'
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,channel,tissue,mouse_id,ksizes,num_hashes,tenx,id,output
16639,10X_P4_7,Spleen,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_7,Spleen_10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=5000/Spleen_10X_P4_7.sig"
45065,10X_P4_5,Kidney,3-M-8,"21,27,33,51",5000,s3://czbiohub-maca/10x_data/10X_P4_5,Kidney_10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=5000/Kidney_10X_P4_5.sig"


In [59]:
samples_cleaned = samples.set_index("id")
samples_cleaned

,channel,tissue,ksizes,num_hashes,tenx,output
id,,,,,,
Spleen_10X_P4_7,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.sig"
Spleen_10X_P7_6,10X_P7_6,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_6,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P7_6.sig"
Kidney_10X_P4_5,10X_P4_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_5.sig"
Kidney_10X_P4_6,10X_P4_6,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_6,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_6.sig"
Kidney_10X_P7_5,10X_P7_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P7_5.sig"


In [60]:
import json

folder = '../sourmash/maca/10x_spleen_kidney'

! mkdir -p $folder
assert samples_cleaned.index.is_unique
samples_cleaned.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": "../../../reflow/sourmash_compute_10x.rf",
		"runs_file": "samples.csv"
	}

with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! head -n 2 $folder/samples.csv $folder/config.json

==> ../sourmash/maca/10x_spleen_kidney/samples.csv <==
id,channel,tissue,ksizes,num_hashes,tenx,output
Spleen_10X_P4_7,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.sig"

==> ../sourmash/maca/10x_spleen_kidney/config.json <==
{"program": "../../../reflow/sourmash_compute_10x.rf", "runs_file": "samples.csv"}

In [ ]:
! aws s3 ls s3://czbiohub-maca/10x_data/10X_P4_7/

In [61]:
import os

for i, row in samples_cleaned.iterrows():
    tenx = row['tenx']
    tissue = row['tissue']
    channel = row['channel']
    dirname = f'/mnt/data/{tissue}_{channel}'
    ! aws s3 sync --dryrun --exclude "*" --include "*bam*" --include "barcodes*" $tenx $dirname

(dryrun) download: s3://czbiohub-maca/10x_data/10X_P4_7/possorted_genome_bam.bam to ../../../../mnt/data/Spleen_10X_P4_7/possorted_genome_bam.bam
(dryrun) download: s3://czbiohub-maca/10x_data/10X_P7_6/barcodes.tsv to ../../../../mnt/data/Spleen_10X_P7_6/barcodes.tsv
(dryrun) download: s3://czbiohub-maca/10x_data/10X_P7_6/possorted_genome_bam.bam to ../../../../mnt/data/Spleen_10X_P7_6/possorted_genome_bam.bam
(dryrun) download: s3://czbiohub-maca/10x_data/10X_P7_6/possorted_genome_bam.bam.bai to ../../../../mnt/data/Spleen_10X_P7_6/possorted_genome_bam.bam.bai
(dryrun) download: s3://czbiohub-maca/10x_data/10X_P4_5/barcodes.tsv to ../../../../mnt/data/Kidney_10X_P4_5/barcodes.tsv
(dryrun) download: s3://czbiohub-maca/10x_data/10X_P4_5/possorted_genome_bam.bam to ../../../../mnt/data/Kidney_10X_P4_5/possorted_genome_bam.bam
(dryrun) download: s3://czbiohub-maca/10x_data/10X_P4_5/possorted_genome_bam.bam.bai to ../../../../mnt/data/Kidney_10X_P4_5/possorted_genome_bam.bam.bai
(dryrun) d

In [64]:
ls /mnt/data/10X*/sourmash_compare

/mnt/data/10X_P4_3/sourmash_compare:
possorted_genome_bam__k21_dna_ignore-abundance=False.csv
possorted_genome_bam__k21_dna_ignore-abundance=True.csv
possorted_genome_bam__k21_protein_ignore-abundance=False.csv
possorted_genome_bam__k21_protein_ignore-abundance=True.csv
possorted_genome_bam__k27_dna_ignore-abundance=False.csv
possorted_genome_bam__k27_dna_ignore-abundance=True.csv
possorted_genome_bam__k27_protein_ignore-abundance=False.csv
possorted_genome_bam__k27_protein_ignore-abundance=True.csv
possorted_genome_bam__k33_dna_ignore-abundance=False.csv
possorted_genome_bam__k33_dna_ignore-abundance=True.csv
possorted_genome_bam__k33_protein_ignore-abundance=False.csv
possorted_genome_bam__k33_protein_ignore-abundance=True.csv
possorted_genome_bam__k51_dna_ignore-abundance=False.csv
possorted_genome_bam__k51_dna_ignore-abundance=True.csv
possorted_genome_bam__k51_protein_ignore-abundance=False.csv
possorted_genome_bam__k51_protein_ignore-abundance=True.csv
possorted_genome_bam__k_dna

In [65]:
ls -lha /mnt/data/10X_P4_3/

total 28G
drwxrwxr-x  3 ubuntu ubuntu 4.0K Oct 13 00:13 ./
drwxr-xr-x 44 ubuntu root   4.0K Oct 23 20:50 ../
-rw-rw-r--  1 ubuntu ubuntu  15M Aug 22  2017 10X_P4_3.mus.cell-gene.csv
-rw-rw-r--  1 ubuntu ubuntu  14G Aug 22  2017 10X_P4_3.tgz
-rw-rw-r--  1 ubuntu ubuntu 2.8K Aug 31  2017 barcodes.tsv
-rw-rw-r--  1 ubuntu ubuntu    0 Oct 13 01:15 csv
-rw-rw-r--  1 ubuntu ubuntu 340K Aug 31  2017 genes.tsv
-rw-rw-r--  1 ubuntu ubuntu 5.4M Aug 31  2017 matrix.mtx
-rw-rw-r--  1 ubuntu ubuntu  615 Aug 22  2017 metrics_summary.csv
-rw-rw-r--  1 ubuntu ubuntu  14G Jun 19 02:34 possorted_genome_bam.bam
-rw-rw-r--  1 ubuntu ubuntu 4.7M Jun 19 02:34 possorted_genome_bam.bam.bai
-rw-rw-r--  1 ubuntu ubuntu 227M Sep 24 19:10 possorted_genome_bam_num_hashes=10000.sig
-rw-rw-r--  1 ubuntu ubuntu    0 Sep 24 09:06 possorted_genome_bam_scaled=100.sig
-rw-rw-r--  1 ubuntu ubuntu  81M Sep 24 08:11 possorted_genome_bam.sig
-rw-rw-r--  1 ubuntu ubuntu 7.4M Sep 15  2017 raw_gene_bc_matrices_h5.h5
drwxrwxr-x 

In [66]:
ls -lah /mnt/data/10X_P4_3/

total 28G
drwxrwxr-x  3 ubuntu ubuntu 4.0K Oct 13 00:13 ./
drwxr-xr-x 44 ubuntu root   4.0K Oct 23 20:50 ../
-rw-rw-r--  1 ubuntu ubuntu  15M Aug 22  2017 10X_P4_3.mus.cell-gene.csv
-rw-rw-r--  1 ubuntu ubuntu  14G Aug 22  2017 10X_P4_3.tgz
-rw-rw-r--  1 ubuntu ubuntu 2.8K Aug 31  2017 barcodes.tsv
-rw-rw-r--  1 ubuntu ubuntu    0 Oct 13 01:15 csv
-rw-rw-r--  1 ubuntu ubuntu 340K Aug 31  2017 genes.tsv
-rw-rw-r--  1 ubuntu ubuntu 5.4M Aug 31  2017 matrix.mtx
-rw-rw-r--  1 ubuntu ubuntu  615 Aug 22  2017 metrics_summary.csv
-rw-rw-r--  1 ubuntu ubuntu  14G Jun 19 02:34 possorted_genome_bam.bam
-rw-rw-r--  1 ubuntu ubuntu 4.7M Jun 19 02:34 possorted_genome_bam.bam.bai
-rw-rw-r--  1 ubuntu ubuntu 227M Sep 24 19:10 possorted_genome_bam_num_hashes=10000.sig
-rw-rw-r--  1 ubuntu ubuntu    0 Sep 24 09:06 possorted_genome_bam_scaled=100.sig
-rw-rw-r--  1 ubuntu ubuntu  81M Sep 24 08:11 possorted_genome_bam.sig
-rw-rw-r--  1 ubuntu ubuntu 7.4M Sep 15  2017 raw_gene_bc_matrices_h5.h5
drwxrwxr-x 